In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import autogluon.core as ag
from autogluon.vision import ObjectDetector

In [ ]:
dataset_train = ObjectDetector.Dataset.from_voc('~/VOCdevkit/VOC2012', splits='train')

In [ ]:
detector = ObjectDetector()

time_limit = 60*5 # 5 minutes
detector.fit(dataset_train, time_limit=time_limit)

In [ ]:
detector.save('face_detector.ag')

# Show results

In [ ]:
image0_path = dataset_test.iloc[0]['image']
result = detector.predict(image0_path).query('predict_score >= 0.7')

In [ ]:
import numpy as np
from PIL import Image

im0 = Image.open(image0_path)
width, height = im0.size
image0 = np.array(im0)

In [ ]:
bboxes=[]
for rois in result['predict_rois']:
    x1, y1 = int(rois['xmin']* width), int(rois['ymin']* height)
    x2, y2 = int(rois['xmax']* width), int(rois['ymax']* height)
    bboxes.append([x1, y1, x2, y2])
    
bboxes = np.array(bboxes)

In [ ]:
scores = result['predict_score'].values
labels , class_names = selected_result['predict_class'].factorize()

In [ ]:
from gluoncv.utils import viz

ax = viz.plot_bbox(image0, bboxes=bboxes, scores=scores, 
                   labels = labels, class_names=class_names)
plt.show()